In [ ]:
!pip install -U optuna
#!pip install -U pfp-api-client
#!pip install pfcc_extras
!pip install -U pfp-api-client matlantis-features
!pip install pfcc-extras-v0.11.1.zip
!pip install pfcc-ase-extras-v0.3.0.zip
#In addition, please install `pfcc_extras`.

In [ ]:
import io
import os
import tempfile

from ase import Atoms
from ase.build import bulk, fcc111, molecule, add_adsorbate
from ase.constraints import ExpCellFilter, StrainFilter
from ase.io import write, read
from ase.io.jsonio import write_json, read_json
from ase.optimize import LBFGS, FIRE
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import optuna
from ase.visualize import view


import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

from pfcc_extras.visualize.view import view_ngl
from pfcc_extras.visualize.ase import view_ase_atoms

print(f"pfp_api_client: {pfp_api_client.__version__}")

# estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL, model_version="latest")
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0, model_version="v3.0.0")
calculator = ASECalculator(estimator)

In [ ]:
def get_opt_energy(atoms, fmax=0.001, opt_mode: str = "normal"):    
    atoms.set_calculator(calculator)
    if opt_mode == "scale":
        opt1 = LBFGS(StrainFilter(atoms, mask=[1, 1, 1, 0, 0, 0]), logfile=None)
    elif opt_mode == "all":
        opt1 = LBFGS(ExpCellFilter(atoms), logfile=None)
    else:
        opt1 = LBFGS(atoms, logfile=None)
    opt1.run(fmax=fmax)
    return atoms.get_total_energy()

In [ ]:
# y = x^2 (0 <= x <= 1)を最小化する例
def objective(trial):
    x = trial.suggest_float("x", 0, 1)
    return x ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=30)
optuna.visualization.plot_optimization_history(study)

In [ ]:
#bulk_atoms = bulk("Pt", cubic=True)
#bulk_atoms.calc = calculator
#E_bulk = get_opt_energy(bulk_atoms, fmax=1e-4, opt_mode="scale")
#E_bulk

In [ ]:
#build structure
def create_slab():
    a = np.mean(np.diag(bulk_atoms.cell))
    slab =  fcc111("Pt", a=a, size=(4, 4, 4), vacuum=40.0, periodic=True)
    slab.calc = calculator
    E_slab = get_opt_energy(slab, fmax=1e-4, opt_mode="normal")
    return slab, E_slab 

#slab, E_slab = create_slab()
#view_ngl(slab, representations=["ball+stick"])

def create_mol():
    mol = molecule("CO")
    mol.calc = calculator
    E_mol = get_opt_energy(mol, fmax=1e-4)
    return mol, E_mol

#mol, E_mol = create_mol()
#view_ngl(mol, representations=["ball+stick"])

def already_slab():
    slab = read("surface.cif")
    slab.calc = calculator
    E_slab = get_opt_energy(slab, fmax=1e-4, opt_mode="normal")
    return slab, E_slab 
slab, E_slab = already_slab()
view_ngl(slab, representations=["ball+stick"])


In [ ]:
def already_mol():
    mol = read("2-ketone.cif")
    mol.calc = calculator
    E_mol = get_opt_energy(mol, fmax=1e-4)
    return mol, E_mol
mol, E_mol = already_mol()
view_ngl(mol, representations=["ball+stick"])
    

In [ ]:
#search ads for big mol
import io

def atoms_to_json(atoms):
    f = io.StringIO()
    write(f, atoms, format="json")
    return f.getvalue()


def json_to_atoms(atoms_str):
    return read(io.StringIO(atoms_str), format="json")

In [ ]:
mol_json_str = atoms_to_json(mol)
mol2 = json_to_atoms(mol_json_str)

print(f"{mol_json_str=}")
view_ngl(mol2, representations=["ball+stick"])

In [ ]:
def objective(trial):
    slab = json_to_atoms(trial.study.user_attrs["slab"])
    E_slab = trial.study.user_attrs["E_slab"]
    
    mol = json_to_atoms(trial.study.user_attrs["mol"])
    E_mol = trial.study.user_attrs["E_mol"]
    
    phi = 180. * trial.suggest_float("phi", -1, 1)
    theta = np.arccos(trial.suggest_float("theta", -1, 1))*180./np.pi
    psi = 180 * trial.suggest_float("psi", -1, 1)
    x_pos = trial.suggest_float("x_pos", 0, 0.5)
    y_pos = trial.suggest_float("y_pos", 0, 0.5)
    z_hig = trial.suggest_float("z_hig", 1, 5)
    xy_position=np.matmul([x_pos,y_pos,0], slab.cell)[:2]
    mol.euler_rotate(phi=phi, theta=theta, psi=psi)
    
    add_adsorbate(slab, mol, z_hig, xy_position)
    E_slab_mol = get_opt_energy(slab, fmax=1e-3)
    
    trial.set_user_attr("structure", atoms_to_json(slab))
    
    return E_slab_mol - E_slab - E_mol


study = optuna.create_study()

slab, E_slab = already_slab()
study.set_user_attr("slab", atoms_to_json(slab))
study.set_user_attr("E_slab", E_slab)

mol, E_mol = already_mol()
study.set_user_attr("mol", atoms_to_json(mol))
study.set_user_attr("E_mol", E_mol)

study.optimize(objective, n_trials=30)
print(f"Best trial is #{study.best_trial.number}")
print(f"    Its adsorption energy is {study.best_value}")
print(f"    Its adsorption position is")
print(f"        phi  : {study.best_params['phi']}")
print(f"        theta: {study.best_params['theta']}")
print(f"        psi. : {study.best_params['psi']}")
print(f"        x_pos: {study.best_params['x_pos']}")
print(f"        y_pos: {study.best_params['y_pos']}")
print(f"        z_hig: {study.best_params['z_hig']}")

In [ ]:
#use optuna
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
slab = json_to_atoms(study.best_trial.user_attrs["structure"])
view_ngl(slab, representations=["ball+stick"])

In [ ]:
os.makedirs("output", exist_ok=True)

fig, axes = plt.subplots(len(study.trials) // 10, 10, figsize=(20, 10))
for trial in study.trials:
    slab = json_to_atoms(trial.user_attrs["structure"])
    write(f"output/{trial.number}.png", slab, rotation="0x,0y,90z")
    ax = axes[trial.number // 10][trial.number % 10]
    ax.imshow(mpimg.imread(f"output/{trial.number}.png"))
    ax.set_axis_off()
    ax.set_title(trial.number)
fig.show()

In [ ]:
slabs = []
for trial in study.trials:
    slab = json_to_atoms(trial.user_attrs["structure"])
    slabs.append(slab)
view_ngl(slabs, representations=["ball+stick"], replace_structure=True)

In [ ]:
import os
from ase.build import molecule
from ase.io import write
import optuna

# —— 1. 分子名称列表 —— #
mol_names = ["2-ketone", "3-ketone", "4-ketone", "5-ketone"] 

# —— 2. 循环处理每一个分子 —— #
for mol_name in mol_names:
    # 2.1 创建输出目录，以分子名称命名
    outdir = os.path.join("results", mol_name)
    os.makedirs(outdir, exist_ok=True)

    slab, E_slab = already_slab()
    study.set_user_attr("slab", atoms_to_json(slab))
    study.set_user_attr("E_slab", E_slab)

    mol, E_mol = already_mol()
    study.set_user_attr("mol", atoms_to_json(mol))
    study.set_user_attr("E_mol", E_mol)

    # 2.4 新建 Optuna study，并存入 user_attrs
    study = optuna.create_study(direction="minimize")
    study.set_user_attr("slab", atoms_to_json(slab))
    study.set_user_attr("E_slab", E_slab)
    study.set_user_attr("mol", atoms_to_json(mol))
    study.set_user_attr("E_mol", E_mol)

    # 2.5 运行优化（可根据需求调整试验次数）
    study.optimize(objective, n_trials=30)

    # 2.6 拿到最优结果
    best = study.best_trial
    params = best.params

    # 2.7 用最优参数构建吸附结构
    best_slab = build_adsorbed_structure(slab, mol, params)

    # 2.8 保存结构文件，直接用分子名称作为文件名
    outfile = os.path.join(outdir, f"best_ads_{mol_name}.cif")
    write(outfile, best_slab)

    # 2.9 打印进度
    print(f"[{mol_name}] 最优能量 {best.value:.4f} eV，结构已保存到 {outfile}")
